Primero importamos pandas y cargamos nuestro dataset desde una ubicación que pueda leerse con read_csv:

In [61]:
import pandas as pd

laptops_dataframe = pd.read_csv("https://raw.githubusercontent.com/rubengr16/MachineLearning/main/Prac1/train_without_outlayers.csv", sep=",")
laptops_dataframe

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,981,Toshiba,Portege Z30-C-1CV,Notebook,13.3,Full HD 1920x1080,Intel Core i5 6200U 2.3GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 7,1.2kg,1195.00
1,996,HP,Spectre 13-V111dx,Ultrabook,13.3,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.11kg,1349.00
2,77,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i7 8550U 1.8GHz,8GB,128GB SSD + 1TB HDD,Intel UHD Graphics 620,Windows 10,2.02kg,855.00
3,103,HP,Envy 13-ad009n,Ultrabook,13.3,IPS Panel Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,256GB SSD,Nvidia GeForce MX150,Windows 10,1.38kg,1119.00
4,946,Dell,Latitude 7280,Ultrabook,12.5,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics,Windows 10,1.36kg,1472.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,994,Dell,Inspiron 5378,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,Windows 10,1.68kg,889.00
877,615,Asus,FX753VD-GC007T (i7-7700HQ/8GB/1TB,Gaming,17.3,Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,8GB,128GB SSD + 1TB HDD,Nvidia GeForce GTX 1050,Windows 10,3kg,1168.00
878,1063,Dell,Inspiron 5567,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,16GB,2TB HDD,AMD Radeon R7 M445,Windows 10,2.32kg,989.99
879,96,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,AMD Radeon R5 M430,Linux,2.2kg,599.90


In [62]:
def to_letter(size: list, num_letter: dict = None):
  if (num_letter is None):
    unique_size = list(set(size))
    unique_size.sort()
    gap_size = int(len(unique_size) / 3)
    num_letter = {unique_size[gap_size]: 'S', unique_size[gap_size * 2]: 'M'}
  dict_keys= list(num_letter.keys())

  for i in range(len(size)):
    if (size[i] < dict_keys[0]):
      size[i] = num_letter[dict_keys[0]]
    elif (size[i] < dict_keys[1]):
      size[i] = num_letter[dict_keys[1]]
    else:
      size[i] = 'L'
  return (size, num_letter)

In [63]:
def parse_inches(laptops_dataframe, num_letter: dict = None):
  inches = []

  for i in range(len(laptops_dataframe)):
    inches.append(int(laptops_dataframe.loc[i, 'Inches']))
  (inches, num_letter) = to_letter(inches, num_letter)
  laptops_dataframe['Inches'] = inches
  return (laptops_dataframe, num_letter)


In [64]:
def get_size(screen_split: list) -> int:
  screen_split = str(screen_split[-1]).split("x")
  
  if (len(screen_split) != 2):
    # Si size no se representa por <num1>x<num2>, no comprendemos el valor
    # y ponemos un 0
    return (0)
  else:
    return int(screen_split[0]) * int(screen_split[1])


def get_resolution(name: str) -> str:
  resolution = ""
  name = name.lower()

  if ("full hd" in name or "fhd" in name):
    resolution = resolution + ' ' + "FHD"
  elif ("4k" in name or "ultra hd" in name or "uhd" in name):
    resolution = resolution + ' ' + "UHD"
  elif ("hd" in name):
    resolution = resolution + ' ' + "HD"
  if ("touchscreen" in name):
    resolution = resolution + ' ' + "TS"
  if ("ips" in name):
    resolution = resolution + ' ' + "IPS"
  return resolution



def parse_screen(laptops_dataframe, num_letter: dict = None):
  size = []

  for i in range(len(laptops_dataframe)):
    size.append(get_size(laptops_dataframe.loc[i,'ScreenResolution'].split()))
    laptops_dataframe.loc[i,'ScreenResolution'] = get_resolution(laptops_dataframe.loc[i, 'ScreenResolution'])
  (size, num_letter) = to_letter(size, num_letter)
  laptops_dataframe['Size'] = size
  return (laptops_dataframe, num_letter)

In [65]:
def parse_ram(laptops_dataframe):
  for i in range(len(laptops_dataframe)):
    laptops_dataframe.loc[i,'Ram'] = int(laptops_dataframe.loc[i,'Ram'][:-2])
  return (laptops_dataframe)

In [66]:
def get_cpu_id(name: str) -> str:
  name = name.lower()

  if ("intel core" in name):
    return name.split()[2]
  if ("intel celeron" in name):
    return "celeron"
  if ("intel pentium" in name):
    return "pentium"
  if ("amd" in name):
    return name.split()[1].split('-')[0]

def parse_cpu(laptops_dataframe):
  speed = []

  for i in range(len(laptops_dataframe)):
    cpu_split = laptops_dataframe.loc[i,'Cpu'].split()
    speed.append(float(cpu_split[-1][:-3]))
    laptops_dataframe.loc[i,'Cpu'] = get_cpu_id(laptops_dataframe.loc[i,'Cpu'])
  laptops_dataframe['Speed'] = speed
  return laptops_dataframe

In [67]:
def get_mem_type(name: str) -> str:
  mem_type = ""
  name = name.lower()

  if ("hdd" in name or "hybrid" in name):
    mem_type = mem_type + ' ' + "HDD"
  if ("ssd" in name or "hybrid" in name):
    mem_type = mem_type + ' ' + "SSD"
  return mem_type


def to_gb(num: int, unit: str) -> int:
  unit = unit.lower()
  if (unit == "gb"):
    return num
  else:
    return num * 1024


def get_total_size(size: str) -> str:
  size = size.split('+')
  size[0] = size[0].split()[0]
  num_size = to_gb(int(size[0][:-2].split('.')[0]), size[0][-2:])
  if (len(size) > 1):
    size[1] = size[1].split()[0]
    num_size += to_gb(int(size[1][:-2].split('.')[0]), size[0][-2:])
  return num_size

def parse_memory(laptops_dataframe):
  mem_type = []

  for i in range(len(laptops_dataframe)):
    mem_type.append(get_mem_type(laptops_dataframe.loc[i,'Memory']))
    laptops_dataframe.loc[i,'Memory'] = get_total_size(laptops_dataframe.loc[i,'Memory'])
  laptops_dataframe['MemoryType'] = mem_type
  return laptops_dataframe

In [68]:
def parse_gpu(laptops_dataframe):
  for i in range(len(laptops_dataframe)):
    laptops_dataframe.loc[i,'Gpu'] = laptops_dataframe.loc[i,'Gpu'].split()[0];
  return (laptops_dataframe)

In [69]:
def parse_weight(laptops_dataframe, num_letter: dict = None):
  weight = []

  for i in range(len(laptops_dataframe)):
    weight.append(float(laptops_dataframe.loc[i, 'Weight'][:-2]))
  (weight, num_letter) = to_letter(weight, num_letter)
  laptops_dataframe['Weight'] = weight
  return (laptops_dataframe, num_letter)

Parseo principal y preprocesado de datos, mediante las funciones creadas anteriormente

In [70]:
def parser(laptops_dataframe, num_letters: list = [None, None, None]):
  (laptops_dataframe, num_letters[0]) = parse_inches(laptops_dataframe, num_letters[0])
  (laptops_dataframe, num_letters[1]) = parse_screen(laptops_dataframe, num_letters[1])
  laptops_dataframe = parse_ram(laptops_dataframe)
  laptops_dataframe = parse_cpu(laptops_dataframe)
  laptops_dataframe = parse_memory(laptops_dataframe)
  laptops_dataframe = parse_gpu(laptops_dataframe)
  (laptops_dataframe, num_letters[2]) = parse_weight(laptops_dataframe, num_letters[2])
  return (laptops_dataframe, num_letters)

(laptops_dataframe, num_letters) = parser(laptops_dataframe)

laptops_dataframe

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,Size,Speed,MemoryType
0,981,Toshiba,Portege Z30-C-1CV,Notebook,M,FHD,i5,4,128,0,Windows 7,S,1195.00,S,2.3,SSD
1,996,HP,Spectre 13-V111dx,Ultrabook,M,FHD TS IPS,i7,8,256,0,Windows 10,S,1349.00,S,2.7,SSD
2,77,Dell,Inspiron 5570,Notebook,L,FHD,i7,8,129,0,Windows 10,M,855.00,S,1.8,HDD SSD
3,103,HP,Envy 13-ad009n,Ultrabook,M,FHD IPS,i7,8,256,0,Windows 10,S,1119.00,S,2.7,SSD
4,946,Dell,Latitude 7280,Ultrabook,M,FHD TS,i5,8,256,0,Windows 10,S,1472.20,S,2.5,SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,994,Dell,Inspiron 5378,2 in 1 Convertible,M,FHD TS,i5,8,256,0,Windows 10,M,889.00,S,2.5,SSD
877,615,Asus,FX753VD-GC007T (i7-7700HQ/8GB/1TB,Gaming,L,FHD,i7,8,129,0,Windows 10,L,1168.00,S,2.8,HDD SSD
878,1063,Dell,Inspiron 5567,Notebook,L,FHD,i7,16,2048,0,Windows 10,L,989.99,S,2.7,HDD
879,96,Dell,Inspiron 3567,Notebook,L,FHD,i7,8,1024,0,Linux,M,599.90,S,2.7,HDD


In [86]:
list(laptops_dataframe.columns)

['LaptopId',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price',
 'Size',
 'Speed',
 'MemoryType']

In [90]:
import plotly.express as px
for column in laptops_dataframe.columns:
  if (column in ["LaptopId", "Product", "Memory", "Speed", "Price"]):
    continue
  fig = px.box(laptops_dataframe, x="Price", y=column, points="all")
  fig.show()

In [23]:
Q1 = laptops_dataframe.Speed.quantile(0.25)
Q3 = laptops_dataframe.Speed.quantile(0.75)
IQR = Q3 - Q1
(Q1, Q3, IQR)
filter = (laptops_dataframe.Speed >= Q1 - 1.5 * IQR) & (laptops_dataframe.Speed <= Q3 + 1.5 *IQR)
laptops_dataframe.loc[filter]
# Q3 = df['AVG'].quantile(0.75)
# IQR = Q3 - Q1    #IQR is interquartile range. 

# filter = (df['AVG'] >= Q1 - 1.5 * IQR) & (df['AVG'] <= Q3 + 1.5 *IQR)
# df.loc[filter]  

,LaptopId,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price,Size,Speed,MemoryType
0,981,Toshiba,Portege Z30-C-1CV,Notebook,M,FHD,i5,4,128,Intel,Windows 7,S,1195.00,S,2.3,SSD
1,996,HP,Spectre 13-V111dx,Ultrabook,M,FHD TS IPS,i7,8,256,Intel,Windows 10,S,1349.00,S,2.7,SSD
2,77,Dell,Inspiron 5570,Notebook,L,FHD,i7,8,129,Intel,Windows 10,M,855.00,S,1.8,HDD SSD
3,103,HP,Envy 13-ad009n,Ultrabook,M,FHD IPS,i7,8,256,Nvidia,Windows 10,S,1119.00,S,2.7,SSD
4,946,Dell,Latitude 7280,Ultrabook,M,FHD TS,i5,8,256,Intel,Windows 10,S,1472.20,S,2.5,SSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,994,Dell,Inspiron 5378,2 in 1 Convertible,M,FHD TS,i5,8,256,Intel,Windows 10,M,889.00,S,2.5,SSD
908,615,Asus,FX753VD-GC007T (i7-7700HQ/8GB/1TB,Gaming,L,FHD,i7,8,129,Nvidia,Windows 10,L,1168.00,S,2.8,HDD SSD
909,1063,Dell,Inspiron 5567,Notebook,L,FHD,i7,16,2048,AMD,Windows 10,M,989.99,S,2.7,HDD
910,96,Dell,Inspiron 3567,Notebook,L,FHD,i7,8,1024,AMD,Linux,M,599.90,S,2.7,HDD


In [60]:
set(laptops_dataframe.Gpu)

{'AMD', 'ARM', 'Intel', 'Nvidia'}

Aplicamos el preprocesado de datos para transformar toda la información obtenida y convertida en el paso anterior

In [71]:
import numpy as np
import sklearn.compose

def transformer_data(laptops_dataframe):
  np_laptops = np.array(laptops_dataframe)
  column_transformer = sklearn.compose.ColumnTransformer(transformers=[
      ("id", "drop", [0]),
      ("Company", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Company))]), [1]),
      # ("Company", "drop", [1]),
      ("product", "drop", [2]),
      ("TypeName", sklearn.preprocessing.OrdinalEncoder(categories=[['Netbook', '2 in 1 Convertible', 'Notebook', 'Gaming', 'Ultrabook', 'Workstation']]), [3]),
      ("Inches", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Inches))]), [4]),
      ("ScreenResolution", sklearn.preprocessing.OrdinalEncoder(categories=[['', ' IPS', ' TS', ' TS IPS', ' HD', ' HD IPS', ' HD TS', ' HD TS IPS', ' FHD', ' FHD IPS', ' FHD TS', ' FHD TS IPS',  ' UHD', ' UHD IPS', ' UHD TS', ' UHD TS IPS']]), [5]),
      # ("Cpu", "drop", [6]),
      ("Cpu", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Cpu))]), [6]),
      ("Ram", sklearn.preprocessing.OrdinalEncoder(), [7]),
      # ("Ram", sklearn.preprocessing.OneHotEncoder(), [7]),
      ("Memory", sklearn.preprocessing.StandardScaler(), [8]),
      # ("Gpu", "drop", [9]),
      # ("OpSys", "drop", [10]),      
      ("Gpu", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Gpu))]), [9]),
      # ("Gpu", "drop", [9]),
      ("OpSys", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.OpSys))]), [10]),
      # ("Weight", "drop", [11]),
      ("Weight", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Weight))]), [11]),
      ("prize", "drop", [12]), 
      # ("Size", "drop", [13]),
      ("Size", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Size))]), [13]),
      ("Speed", sklearn.preprocessing.StandardScaler(), [14]),
      ("MemoryType", sklearn.preprocessing.OrdinalEncoder(categories=[['', ' HDD', ' SSD', ' HDD SSD']]), [15])
  ]);
  return column_transformer.fit_transform(np_laptops)

def transformer_target(laptops_dataframe):
  np_laptops = np.array(laptops_dataframe)
  column_transformer = sklearn.compose.ColumnTransformer(transformers=[
      ("id", "drop", [0]),
      ("Company", "drop", [1]),
      ("product", "drop", [2]),
      ("TypeName", "drop", [3]),
      ("Inches", "drop", [4]),
      ("ScreenResolution", "drop", [5]),
      ("Cpu", "drop", [6]),
      ("Ram", "drop", [7]),
      ("Memory", "drop", [8]),
      ("Gpu", "drop", [9]),
      ("OpSys", "drop", [10]),
      ("Weight", "drop", [11]),
      ("price", "passthrough", [12]),
      ("Size", "drop", [13]),
      ("Speed", "drop", [14]),
      ("MemoryType","drop", [15])
  ]);
  return column_transformer.fit_transform(np_laptops)

def transformer_test(test_dataframe, laptops_dataframe):
  np_test = np.array(test_dataframe)
  column_transformer = sklearn.compose.ColumnTransformer(transformers=[
      ("id", "drop", [0]),
      ("Company", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Company))]), [1]),
      # ("Company", "drop", [1]),
      ("product", "drop", [2]),
      ("TypeName", sklearn.preprocessing.OrdinalEncoder(categories=[['Netbook', '2 in 1 Convertible', 'Notebook', 'Gaming', 'Ultrabook', 'Workstation']]), [3]),
      ("Inches", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Inches))]), [4]),
      ("ScreenResolution", sklearn.preprocessing.OrdinalEncoder(categories=[['', ' IPS', ' TS', ' TS IPS', ' HD', ' HD IPS', ' HD TS', ' HD TS IPS', ' FHD', ' FHD IPS', ' FHD TS', ' FHD TS IPS',  ' UHD', ' UHD IPS', ' UHD TS', ' UHD TS IPS']]), [5]),
      # ("Cpu", "drop", [6]),
      ("Cpu", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Cpu))]), [6]),
      ("Ram", sklearn.preprocessing.OrdinalEncoder(), [7]),
      # ("Ram", sklearn.preprocessing.OneHotEncoder(), [7]),
      ("Memory", sklearn.preprocessing.StandardScaler(), [8]),
      # ("Gpu", "drop", [9]),
      # ("OpSys", "drop", [10]),      
      ("Gpu", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Gpu))]), [9]),
      # ("Gpu", "drop", [9]),
      ("OpSys", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.OpSys))]), [10]),
      # ("Weight", "drop", [11]),
      ("Weight", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Weight))]), [11]),
      # ("Size", "drop", [12]),
      ("Size", sklearn.preprocessing.OneHotEncoder(categories=[list(set(laptops_dataframe.Size))]), [12]),
      ("Speed", sklearn.preprocessing.StandardScaler(), [13]),
      ("MemoryType", sklearn.preprocessing.OrdinalEncoder(categories=[['', ' HDD', ' SSD', ' HDD SSD']]), [14])
  ]);
  return column_transformer.fit_transform(np_test)

In [72]:
from sklearn.linear_model import LinearRegression

data = transformer_data(laptops_dataframe)
target = transformer_target(laptops_dataframe)
reg = LinearRegression().fit(data, target)

In [73]:
test_dataframe = pd.read_csv("https://raw.githubusercontent.com/rubengr16/MachineLearning/main/Prac1/test.csv", sep=",")

num_letters = [None, None, None, None]
(test_dataframe, num_letters) = parser(test_dataframe, num_letters)

test = transformer_test(test_dataframe, laptops_dataframe)

In [74]:
prediction = reg.predict(test)
prediction_dataframe = pd.DataFrame(prediction)
ids = test_dataframe.iloc[:,0]
submit = pd.concat([ids,prediction_dataframe], axis=1)
submit.columns=['LaptopId','Price']
submit.to_csv("submitLinearGpu.csv", index=False)